In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Análise de Dados 

Essa é uma pequena história sobre como a ánalise de dados pode ajudar a entender como funciona o faturamento de um comércio, como contribuir para uma tomada de decisões e como tornar mais competitivo aquele comércio que faz o uso da tecnologia ao seu favor. 

## Base de dados 

O comerciante me enviou os dados de venda 2024 de janeiro até o mês de agosto, porém a planilha está com erros valores, valores faltantes e até mesmo a conversão está como arquivo .xlsx porém os seus dados estão em uma estrutura .csv. Para isso vamos tratar os erros desde a conversão correta do tipo do arquivo até que ele esteja pronto para ser analisado.

## Objetivo

Nosso amigo comerciante pediu para que tratassemos os erros da planilha e pudessemos analisar o seus dados permitindo extrair insights importantes para tomadas de decisões para o seu comércio, nesse projeto vamos aplicar data wrangling, análise de dados, Visualização de Dados e BI.

# Etapa 1 

Converter o arquivo que está em .xslx para .csv


In [4]:
# Convertend o arquivo tipo .xsls para .csv para manipulaçao e tratamento de dados 

xslx = pd.read_excel('./DataSet/OnlineSalesData-NaoTratado.xlsx')

xslx.to_csv('./DataSet/OnlineSalesData-NaoTratado.csv', index='False', sep=';', encoding='utf-8')


In [6]:
# Visualizando a tabela em forma de DataFrame pelo Pandas 

df = pd.read_csv('./DataSet/OnlineSalesData-NaoTratado.csv', sep=';')
df.head()

,Unnamed: 0,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price ($),Total Revenue ($),Region,Payment Method
0,0,10001,01e01e2024,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,1,10002,02e01e2024,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,2,10003,03e01e2024,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,3,10004,04e01e2024,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,4,Null,05e01e2024,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


# Etapa 2 

Agora precisamos tratar os dados, uma impressão rápida que tenho é que tem uma coluna sem referência na tabela e sem nome também. O nome da coluna é 'Unnamed: 0' e ela será removida, depois vamos tratar os dados faltantes e irregulares.

In [ ]:
# Removendo a coluna chamada Unnamed: 0 

df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# Agora que a coluna foi removida vamos começar a trabalhar com as demais colunas, perceba que ao usar df.head ele pode truncar a visualização dos dados para economizar espaço

df.head(n=240)


In [ ]:
# Para isso é importante que você possa ter a opção de ver os todos os dados a partir de seguinte configuração

# Configurar o pandas para exibir todas as colunas

pd.set_option('display.max_columns', None)

# Configurar o pandas para exibir todas as linhas

pd.set_option('display.max_rows', None)

df.head(n=240)

In [ ]:
# Ao analisarmos a tabela agora com mais calma podemos observar que a Coluna ID transaction está com os dados irregulares e apresentando erros, vamos trata-los

# Calculando o numero total de linhas

num_linhas = len(df)
print(num_linhas)

# Criando um range para sobreescrever o ID irregulares sem perder os ID originais já identificados

ids_novos = range(10001, 10001 + num_linhas)

# Atribuir a lista de IDs à coluna 'Transaction ID'

df['Transaction ID'] = list(ids_novos)
df.head(n=240)


# Coluna Date

Iremos tratar agora a coluna Date para que tenhamos as datas definidas corretamentes, iremos remover a letra "e" e por a data no estilo americano

In [ ]:
# Remoção da letra "e" e alterando os dados do tipo string para datetime no estilo americano 

df['Date'] = df['Date'].str.replace('e', '')
df['Date'] = pd.to_datetime(df['Date'], format='%d%m%Y').dt.strftime('%m/%d/%Y')
df.head()

# Colunas Unit Price e Total Revenue

As colunas em questão ao serem registrados como valor monetário na tabela, sofreram alterações que impactaram o resultado real do valor das vendas. Nessa etapa precisamos corrigir o valor monetário para dolar e depois transformar o tipo de dado para float. Com isso terminamos a etapa de limpeza e tratamento de dados. Após isso exportaremos o DataFrame para uma planilha .csv

In [ ]:
# Converter as colunas para string antes de remover o símbolo de Real Brasileiro 'R$'

df['Unit Price ($)'] = df['Unit Price ($)'].astype(str).str.replace('R$', '')
df['Total Revenue ($)'] = df['Total Revenue ($)'].astype(str).str.replace('R$', '')

# Converter as colunas limpas para float

df['Unit Price ($)'] = df['Unit Price ($)'].astype(float)
df['Total Revenue ($)'] = df['Total Revenue ($)'].astype(float)


In [ ]:
# Visualizando os dados após o data wrangling 

df.head()

In [ ]:
# Exportando os dados tratados para uma tabela .csv 
df.to_csv('./DataSet/OnlineSalesData-Tratado.csv', index=False)

# Visualização dos Dados

Após o metódo data wrangling, que por sua vez corrigiu tipos de dados corretamente, tratou erros, eliminou valores nulos e adequou a tabela para análise de dados, podemos seguir em frente com a visualização de dados e em seguida com o BI para tomarmos decisões com base nos dados tratados. 

# Tendências de Vendas ao Longo do Tempo

In [ ]:
# Certifique-se de que a coluna 'Date' é do tipo datetime
df['Date'] = pd.to_datetime(df['Date'])

# Agora você pode agrupar por mês usando o acessor .dt
vendas_tempo = df.groupby(df['Date'].dt.to_period('M')).agg({'Total Revenue ($)':'sum'})

# Convertendo o índice para datetime novamente para plotagem
vendas_tempo.index = vendas_tempo.index.to_timestamp()

# Criando o gráfico de linhas
vendas_tempo.plot(kind='line', figsize=(10, 5))
plt.title('Tendências de Vendas ao Longo do Tempo')
plt.xlabel('Data')
plt.ylabel('Receita Total ($)')
plt.show()


# Receita por Categoria de Produto

In [ ]:
# Agrupando os dados por categoria de produto e somando a receita
categoria_receita = df.groupby('Product Category')['Total Revenue ($)'].sum()

# Criando o gráfico de barras
categoria_receita.plot(kind='bar', figsize=(10, 5))
plt.title('Receita por Categoria de Produto')
plt.xlabel('Categoria de Produto')
plt.ylabel('Receita Total ($)')
plt.show()

# Distribuição de Métodos de Pagamento

In [ ]:
# Contando a frequência de cada método de pagamento
metodos_pagamento = df['Payment Method'].value_counts()

# Criando o gráfico de pizza
metodos_pagamento.plot(kind='pie', autopct='%1.1f%%', figsize=(7, 7))
plt.title('Distribuição de Métodos de Pagamento')
plt.ylabel('')
plt.show()

# Vendas por Região

In [ ]:
# Agrupando os dados por região e somando a receita
vendas_regiao = df.groupby('Region')['Total Revenue ($)'].sum()

# Criando o gráfico de barras
vendas_regiao.plot(kind='bar', figsize=(10, 5))
plt.title('Vendas por Região')
plt.xlabel('Região')
plt.ylabel('Receita Total ($)')
plt.show()

# Performance top 10 Produtos Individuais mais vendidos

In [ ]:
# Calculando a receita total por produto
df['Total Revenue ($)'] = df['Units Sold'] * df['Unit Price ($)']

# Agrupando os dados por nome do produto e somando a receita total
produtos_rentaveis = df.groupby('Product Name')['Total Revenue ($)'].sum().sort_values(ascending=False)

# Pegando os 10 produtos mais rentáveis
top_10_rentaveis = produtos_rentaveis.head(10)

# Criando o gráfico de barras
top_10_rentaveis.plot(kind='bar', figsize=(10, 5))
plt.title('Top 10 Produtos Mais Rentáveis')
plt.xlabel('Nome do Produto')
plt.ylabel('Receita Total ($)')
plt.show()

# Top 10 produtos com maior valor de vendas agregado

In [ ]:
# Agrupando os dados por região e nome do produto, e somando o valor agregado do produto
produtos_vendidos_regiao = df.groupby(['Region', 'Product Name'])['Total Revenue ($)'].sum().reset_index()

# Criando um DataFrame para armazenar os top 10 produtos mais vendidos por região por valor agregado
top_produtos_regiao = pd.DataFrame()

for regiao in produtos_vendidos_regiao['Region'].unique():
    top_produtos = produtos_vendidos_regiao[produtos_vendidos_regiao['Region'] == regiao].sort_values(by='Total Revenue ($)', ascending=False).head(10)
    top_produtos_regiao = pd.concat([top_produtos_regiao, top_produtos])

# Pivotando os dados para melhor visualização
pivot_top_produtos_regiao = top_produtos_regiao.pivot(index='Product Name', columns='Region', values='Total Revenue ($)').fillna(0)

# Criando o gráfico de barras para cada região
for regiao in pivot_top_produtos_regiao.columns:
    pivot_top_produtos_regiao[regiao].sort_values().plot(kind='barh', figsize=(10, 5))
    plt.title(f'Top 10 Produtos Mais Vendidos em {regiao}')
    plt.xlabel('Quantidade Vendida')
    plt.ylabel('Nome do Produto')
    plt.show()

# Análise de Dados

##  “Tendências de Vendas ao Longo do Tempo”

Analisando o gráfico “Tendências de Vendas ao Longo do Tempo”, observamos um padrão distinto nas vendas da empresa ao longo dos meses de janeiro a agosto de 2024. O início do ano mostra um pico significativo em janeiro, indicando um forte desempenho de vendas, possivelmente impulsionado por promoções de Ano Novo ou pela demanda acumulada do período de festas.

No entanto, há uma queda notável em fevereiro, que se aprofunda drasticamente em março. Este declínio pode ser atribuído a vários fatores, como a sazonalidade do mercado ou o esgotamento do impulso das vendas pós-férias. A tendência decrescente continua até maio, sugerindo a necessidade de uma análise mais aprofundada para identificar as causas subjacentes — seja uma mudança no comportamento do consumidor, aumento da concorrência ou outros fatores externos.

Junho marca o ponto mais baixo das vendas, o que pode ser uma oportunidade para a empresa revisar suas estratégias de marketing e vendas. A recuperação modesta em julho e agosto indica que as ações corretivas podem estar começando a surtir efeito, mas ainda há um longo caminho a percorrer para retornar aos números de janeiro.

Recomendações:

Investigar as causas do declínio nas vendas entre fevereiro e junho.
Avaliar a eficácia das campanhas de marketing e promoções atuais.
Considerar estratégias de diversificação de produtos ou mercados para mitigar a sazonalidade.
Monitorar de perto a recuperação nas vendas nos meses de verão para garantir que o crescimento seja sustentável.
Esta análise inicial destaca a importância de entender as tendências de vendas e estar preparado para adaptar as estratégias de negócios conforme necessário. Estou à disposição para analisar os próximos gráficos e fornecer mais insights.



## Vendas por região 

Analisando o gráfico “Receita por Categoria de Produto”, podemos observar claramente que a categoria Electronics se destaca como a mais lucrativa, gerando a maior receita entre todas as categorias listadas. Isso sugere uma forte demanda do mercado ou uma margem de lucro significativa nessa categoria, o que pode ser resultado de lançamentos de novos produtos, inovações tecnológicas ou uma estratégia de preços bem-sucedida.

As categorias Home Appliances e Beauty Products também mostram um bom desempenho, indicando que esses segmentos são áreas importantes para o negócio. Por outro lado, as categorias Books, Clothing e Sports apresentam uma receita menor, o que pode indicar uma saturação no mercado, uma competição mais acirrada ou uma necessidade de revisão na estratégia de vendas para esses produtos.

Recomendações:

Focar em estratégias de expansão para a categoria de Electronics, explorando novos mercados ou segmentos de clientes.
Analisar as operações e estratégias de marketing para as categorias Home Appliances e Beauty Products para continuar o crescimento.
Reavaliar a abordagem de mercado para as categorias Books, Clothing e Sports, buscando inovações ou parcerias estratégicas que possam impulsionar as vendas.
Considerar pesquisas de mercado para entender melhor as preferências dos consumidores e ajustar o portfólio de produtos conforme necessário.
Essa análise de receita por categoria é fundamental para entender onde concentrar esforços e recursos para maximizar o retorno sobre o investimento. Estou à disposição para continuar a análise com as próximas imagens.

## Top 10 produtos mais rentáveis

Analisando o gráfico “Top 10 Produtos Mais Rentáveis”, vemos que a Canon EOS R5 Camera lidera o ranking, indicando uma alta lucratividade, provavelmente devido a uma combinação de preço elevado e boa quantidade de vendas. Produtos tecnológicos como MacBook Pro 16-inch, iPhone 14 Pro, e LG OLED TV também estão entre os mais rentáveis, o que reforça a tendência de que dispositivos eletrônicos de alto valor agregado são significativos geradores de receita para a empresa.

Produtos como Peloton Bike, Roomba i7+, e Garmin Forerunner 945 destacam-se também, sugerindo que há um mercado robusto para produtos de estilo de vida e fitness de alta gama. A presença de um monitor de jogos da Samsung indica que o segmento de gaming é outro contribuidor importante para a rentabilidade.

Recomendações:

Manter o foco em produtos de tecnologia de alto valor, pois eles continuam a ser os mais rentáveis.
Explorar oportunidades de cross-selling e upselling com acessórios e serviços relacionados aos produtos top de linha.
Investir em marketing e promoções para os produtos de estilo de vida e fitness, aproveitando o interesse crescente nesse segmento.
Monitorar as tendências do mercado de gaming, considerando a expansão do portfólio de produtos para atender à demanda dos consumidores.
Esses insights são cruciais para direcionar as estratégias de produto e marketing da empresa, visando maximizar a rentabilidade. Aguardo as próximas imagens para continuar a análise.

## Top 10 Produtos Mais Vendidos em cada Região 

### Ásia 

Analisando o gráfico “Top 10 Produtos Mais Vendidos em Asia”, notamos uma variedade interessante de produtos que capturaram a atenção do mercado asiático. Produtos de tecnologia e fitness, como Peloton Bike, Garmin Fenix 6X Pro, e Fitbit Versa 3, estão entre os mais vendidos, o que pode refletir um crescente interesse por saúde e bem-estar na região.

Além disso, itens como Babolat Pure Drive Tennis Racket e Nike Air Force 1 mostram uma forte presença no mercado de esportes e moda, respectivamente. A diversidade dos produtos mais vendidos sugere que os consumidores asiáticos têm um amplo leque de interesses, desde tecnologia até fitness e moda.

Recomendações:

Capitalizar sobre a tendência de fitness e saúde, expandindo a linha de produtos relacionados e explorando parcerias com influenciadores nesse nicho.
Aumentar o investimento em marketing digital para produtos de tecnologia, aproveitando a alta conectividade e adoção de dispositivos móveis na Ásia.
Desenvolver campanhas de marketing localizadas que ressoem com os interesses culturais e estilos de vida dos consumidores asiáticos.
Monitorar de perto as mudanças nas preferências dos consumidores, adaptando rapidamente o portfólio de produtos para atender às demandas emergentes.



### Europe 

Analisando o gráfico “Top 10 Produtos Mais Vendidos na Europa”, percebemos que os produtos de tecnologia e eletrodomésticos dominam as vendas. Produtos como LG OLED TV, Roomba i7+, e iPhone 14 Pro estão entre os mais vendidos, o que reflete a alta demanda por dispositivos eletrônicos e gadgets inteligentes na Europa. A presença de itens como Dyson Supersonic Hair Dryer e De’Longhi Magnifica Espresso Machine sugere que os consumidores europeus também valorizam produtos de alta qualidade para uso pessoal e doméstico.

Recomendações:

Continuar investindo em produtos de tecnologia e eletrodomésticos, pois eles têm um desempenho forte no mercado europeu.
Desenvolver campanhas de marketing que enfatizem a qualidade, durabilidade e inovação tecnológica dos produtos.
Explorar parcerias com marcas de luxo europeias para produtos de uso pessoal, aproveitando a preferência dos consumidores por itens premium.
Avaliar a possibilidade de expandir a linha de produtos para casa inteligente, dada a popularidade de itens como aspiradores de pó robóticos e monitores de jogos.


### North America

Analisando o gráfico “Top 10 Produtos Mais Vendidos em North America”, vemos que a Canon EOS R5 Camera está no topo da lista, o que indica uma forte demanda por equipamentos fotográficos de alta qualidade na América do Norte. O MacBook Pro 16-inch e o iPhone 14 Pro também estão entre os mais vendidos, reforçando a popularidade dos produtos da Apple na região.

Produtos como HP Spectre x360 Laptop, Samsung Galaxy Tab S8, e Microsoft Surface Laptop 4 mostram que há um mercado significativo para dispositivos móveis e computadores pessoais de alto desempenho. A presença de eletrodomésticos como Roomba i7+ e Dyson Supersonic Hair Dryer sugere que os consumidores estão investindo em conveniência e produtos de cuidados pessoais premium.

Recomendações para a diretoria:

Fortalecer a presença no segmento de tecnologia, especialmente com produtos de marcas reconhecidas como Apple e Samsung.
Explorar o mercado de fotografia com mais opções de câmeras e acessórios, dada a liderança da Canon EOS R5 Camera.
Considerar o desenvolvimento de pacotes e ofertas que combinem dispositivos móveis e acessórios para atrair consumidores que buscam uma experiência integrada.
Avaliar a expansão da linha de produtos para casa inteligente, já que itens como aspiradores robóticos estão se mostrando populares.

